In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('df_modif.csv', encoding='latin-1', sep=',')

In [3]:
df = pd.DataFrame(data)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21187 entries, 0 to 21186
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   estacion  21187 non-null  int64  
 1   Datetime  21187 non-null  object 
 2   DV        21187 non-null  float64
 3   HR        21187 non-null  float64
 4   P         21187 non-null  float64
 5   PB        21187 non-null  float64
 6   RS        21187 non-null  float64
 7   RU        18307 non-null  float64
 8   TEMP      21187 non-null  float64
 9   VV        21187 non-null  float64
 10  BEN       21187 non-null  float64
 11  CH4       21187 non-null  float64
 12  CO        21187 non-null  float64
 13  EBE       21187 non-null  float64
 14  NMHC      21187 non-null  float64
 15  NO        21187 non-null  float64
 16  NO2       21187 non-null  float64
 17  NOX       21187 non-null  float64
 18  O3        21187 non-null  float64
 19  PM10      21187 non-null  float64
 20  PM25      21187 non-null  fl

In [5]:
media = df['RU'].mean()

# Rellena los NaN con la media
df['RU'].fillna(media, inplace=True)

In [6]:
df.isnull().sum()

estacion    0
Datetime    0
DV          0
HR          0
P           0
PB          0
RS          0
RU          0
TEMP        0
VV          0
BEN         0
CH4         0
CO          0
EBE         0
NMHC        0
NO          0
NO2         0
NOX         0
O3          0
PM10        0
PM25        0
SO2         0
TCH         0
TOL         0
dtype: int64

## NORMALIZAR

In [7]:
# quitamos la variable del tipo object (string)
variables = df.drop(["Datetime"], axis=1, inplace=False)

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()

In [10]:
df_transformed = scaler.fit_transform(variables._get_numeric_data())

In [11]:
print(df_transformed)

[[-1.69445476 -0.51518869  0.00656204 ...  3.87963186  1.47847981
   0.75554379]
 [-1.69445476  1.05976054  0.22500006 ...  4.50262185  1.52094627
   2.23568883]
 [-1.69445476 -0.28685032 -0.28954283 ...  3.87963186  1.37939139
   1.6174004 ]
 ...
 [ 1.33573896 -1.02455889 -0.9885445  ... -1.18216183 -0.87840892
  -0.10631282]
 [ 1.33573896 -1.15336514 -0.88175258 ... -1.18216183 -0.87840892
  -0.4997691 ]
 [ 1.33573896 -1.14165549 -0.88175258 ... -1.10428808 -0.87840892
  -0.40608903]]


## DIVISION DATOS

In [12]:
X = df.iloc[:,2:10]
y = df.iloc[:,10:24]

In [13]:
# Importamos el módulo de StratifiedKfold
from sklearn.model_selection import StratifiedKFold
# Importamos el módulo de cross_val_score
from sklearn.model_selection import cross_val_predict
# Importamos el módulo para realizar la partición entre train y test
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X_new= SelectPercentile(chi2, percentile=10)

NameError: name 'SelectPercentile' is not defined

In [16]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.5)
X_new2= selector.fit_transform(X, y)
X_new2

array([[ 55.5       ,  56.63636364,   0.        , ..., 104.33333333,
          4.66666667,   4.5       ],
       [190.        ,  60.72727273,   0.        , ..., 105.        ,
          4.66666667,   4.        ],
       [ 75.        ,  51.09090909,   0.        , ..., 108.        ,
          4.66666667,   4.3       ],
       ...,
       [ 12.        ,  38.        ,   0.        , ..., 335.        ,
          0.        ,  27.7       ],
       [  1.        ,  40.        ,   0.        , ..., 333.        ,
          0.        ,  29.1       ],
       [  2.        ,  40.        ,   0.        , ..., 330.        ,
          0.        ,  29.5       ]])

In [27]:
X_new2.shape

(21187, 7)

In [26]:
variables2 = selector.get_support(indices=True)

In [28]:
print("Variables modelo VarianceThreshold:")
for final_variables in variables2:
        print(X.columns.values[final_variables])

Variables modelo VarianceThreshold:
DV
HR
P
PB
RS
RU
TEMP


## MODELOS

### ARBOL DE DECISION

- arbol decision con mse

In [15]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [105]:
regression_tree = DecisionTreeRegressor(max_depth=15,random_state=42)

In [106]:
regression_tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=15, random_state=42)

In [107]:
predictions = regression_tree.predict(X_test)

In [108]:
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

Mean Squared Error: 104.42178951028428


- arbol decisión con mae

In [16]:
from sklearn.metrics import mean_absolute_error

In [110]:
mae = mean_absolute_error(y_test, predictions)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 3.5486681582743076


- arbol decision con R2

In [17]:
from sklearn.metrics import r2_score

In [112]:
r2 = r2_score(y_test, predictions)
print("Coeficiente de Determinación (R²):", r2)

Coeficiente de Determinación (R²): 0.4824863339361824


### REGRESION LINEAL MULTIPLE

In [113]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [114]:
# Crear un modelo de regresión lineal múltiple
modelo = LinearRegression()

In [115]:
modelo.fit(X_train, y_train)

LinearRegression()

In [116]:
# Realizar predicciones en los datos de prueba
predicciones = modelo.predict(X_test)

In [120]:
mse = mean_squared_error(y_test, predicciones)
r2 = r2_score(y_test, predicciones)
mae = mean_absolute_error(y_test, predicciones)

In [121]:
# Imprimir resultados
print("Mean Squared Error (MSE):", mse)
print("Coeficiente de Determinación (R2):", r2)
print("Mean Absolute Error (MAE):", mae)

# Si deseas ver los coeficientes y la intercepción del modelo, puedes hacerlo así:
print("Coeficientes:", modelo.coef_)
print("Intercepción:", modelo.intercept_)

Mean Squared Error (MSE): 130.3814286945822
Coeficiente de Determinación (R2): 0.29623910646551727
Mean Absolute Error (MAE): 4.791345032395539
Coeficientes: [[ 3.62332905e-04 -4.41979835e-03 -3.21819745e-03  8.60350253e-03
  -7.90401530e-04  3.68469750e-03 -1.35290752e-02 -1.43495448e-01]
 [ 1.73311070e-03 -1.01950839e-03 -6.13118830e-03  2.70832403e-03
  -1.14891856e-03  3.98466605e-02  9.23014612e-03  3.29037911e-02]
 [ 9.24246652e-05 -1.60084003e-03 -1.91172103e-03  3.44246736e-03
  -3.56786469e-04  1.86105447e-03 -4.54922754e-03 -7.92817467e-02]
 [ 1.36492530e-03 -4.83977067e-03 -4.05643058e-03  3.58652419e-03
  -1.84430215e-03 -2.39706519e-03  1.95547925e-02 -2.14012178e-01]
 [ 1.18150526e-04 -3.09841750e-04 -9.12401758e-04  1.97330092e-04
  -1.46939301e-04  1.64028081e-03  1.36573359e-03 -9.51450629e-03]
 [ 3.39456959e-02 -2.08185919e-01 -1.31536780e-01  7.28660691e-01
  -5.13027774e-02  1.62936922e-01 -4.72906791e-01 -9.97461280e+00]
 [ 3.19262792e-02 -4.26203503e-01  1.6642677

In [15]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [21]:
modelo_arbol = DecisionTreeRegressor(max_depth=15)  # Puedes ajustar la profundidad máxima según tus necesidades

# Entrenar el modelo para cada variable objetivo
r2_scores = {}  # Un diccionario para almacenar los resultados R^2

for variable_objetivo in y.columns:
    y_actual = y[variable_objetivo]  # Selecciona la variable objetivo actual
    modelo_arbol.fit(X, y_actual)  # Entrena el modelo para la variable objetivo actual
    y_pred = modelo_arbol.predict(X)  # Realiza predicciones
    
    # Calcula el coeficiente de determinación (R^2) para la variable objetivo actual
    r2 = r2_score(y_actual, y_pred)
    r2_scores[variable_objetivo] = r2

    # Imprime el resultado R^2 para la variable objetivo actual
    print(f'R^2 para {variable_objetivo}: {r2}')

R^2 para BEN: 0.9370466900307285
R^2 para CH4: 0.8724200124283692
R^2 para CO: 0.8678778295619252
R^2 para EBE: 0.8766627393734948
R^2 para NMHC: 0.8188532036816474
R^2 para NO: 0.9375312865829911
R^2 para NO2: 0.911411823626744
R^2 para NOX: 0.9399169331266234
R^2 para O3: 0.9458242130042798
R^2 para PM10: 0.8928075793737691
R^2 para PM25: 0.8959673724550885
R^2 para SO2: 0.7791308319627568
R^2 para TCH: 0.8760227677263429
R^2 para TOL: 0.8890861137184256


In [23]:
from sklearn.linear_model import LinearRegression

In [24]:
modelo_lineal_multiple = LinearRegression()  # Puedes ajustar la profundidad máxima según tus necesidades

# Entrenar el modelo para cada variable objetivo
r2_scores = {}  # Un diccionario para almacenar los resultados R^2

for variable_objetivo in y.columns:
    y_actual = y[variable_objetivo]  # Selecciona la variable objetivo actual
    modelo_lineal_multiple.fit(X, y_actual)  # Entrena el modelo para la variable objetivo actual
    y_pred = modelo_lineal_multiple.predict(X)  # Realiza predicciones
    
    # Calcula el coeficiente de determinación (R^2) para la variable objetivo actual
    r2 = r2_score(y_actual, y_pred)
    r2_scores[variable_objetivo] = r2

    # Imprime el resultado R^2 para la variable objetivo actual
    print(f'R^2 para {variable_objetivo}: {r2}')

R^2 para BEN: 0.4749444035676146
R^2 para CH4: 0.19236341374029642
R^2 para CO: 0.35130050033796156
R^2 para EBE: 0.056953774287627934
R^2 para NMHC: 0.06742061347798312
R^2 para NO: 0.3403702828364046
R^2 para NO2: 0.4776186854151818
R^2 para NOX: 0.41750596604671264
R^2 para O3: 0.686290144735395
R^2 para PM10: 0.18251240585494832
R^2 para PM25: 0.22134774439713567
R^2 para SO2: 0.1526993323289748
R^2 para TCH: 0.18059067646217197
R^2 para TOL: 0.2929717746833065


In [25]:
from sklearn.neighbors import KNeighborsRegressor

modelo_knn = KNeighborsRegressor(n_neighbors=5)  # Puedes ajustar la profundidad máxima según tus necesidades

# Entrenar el modelo para cada variable objetivo
r2_scores = {}  # Un diccionario para almacenar los resultados R^2

for variable_objetivo in y.columns:
    y_actual = y[variable_objetivo]  # Selecciona la variable objetivo actual
    modelo_knn.fit(X, y_actual)  # Entrena el modelo para la variable objetivo actual
    y_pred = modelo_knn.predict(X)  # Realiza predicciones
    
    # Calcula el coeficiente de determinación (R^2) para la variable objetivo actual
    r2 = r2_score(y_actual, y_pred)
    r2_scores[variable_objetivo] = r2

    # Imprime el resultado R^2 para la variable objetivo actual
    print(f'R^2 para {variable_objetivo}: {r2}')

R^2 para BEN: 0.7957334743296568
R^2 para CH4: 0.8034180729642652
R^2 para CO: 0.6408602731476873
R^2 para EBE: 0.5912409097090705
R^2 para NMHC: 0.7551650147646324
R^2 para NO: 0.7225224864169516
R^2 para NO2: 0.733406571586103
R^2 para NOX: 0.7405975849233231
R^2 para O3: 0.870699392594029
R^2 para PM10: 0.7808250373046114
R^2 para PM25: 0.7858891498067984
R^2 para SO2: 0.6513976852221203
R^2 para TCH: 0.7996407631694548
R^2 para TOL: 0.7095880262698417


In [26]:
from sklearn.ensemble import RandomForestRegressor

In [27]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenar el modelo para cada variable objetivo
r2_scores = {}  # Un diccionario para almacenar los resultados R^2

for variable_objetivo in y.columns:
    y_actual = y[variable_objetivo]  # Selecciona la variable objetivo actual
    rf_model.fit(X, y_actual)  # Entrena el modelo para la variable objetivo actual
    y_pred = rf_model.predict(X)  # Realiza predicciones
    
    # Calcula el coeficiente de determinación (R^2) para la variable objetivo actual
    r2 = r2_score(y_actual, y_pred)
    r2_scores[variable_objetivo] = r2

    # Imprime el resultado R^2 para la variable objetivo actual
    print(f'R^2 para {variable_objetivo}: {r2}')

R^2 para BEN: 0.9680080646736083
R^2 para CH4: 0.9801513225401025
R^2 para CO: 0.9368667165940533
R^2 para EBE: 0.9258208337611615
R^2 para NMHC: 0.9703693431955582
R^2 para NO: 0.9595000477779396
R^2 para NO2: 0.9636589710437772
R^2 para NOX: 0.96388590012562
R^2 para O3: 0.9812370131164795
R^2 para PM10: 0.9674231359103795
R^2 para PM25: 0.9709780990609391
R^2 para SO2: 0.9383234568774728
R^2 para TCH: 0.9796494045161149
R^2 para TOL: 0.9536574362447438


In [28]:
from sklearn.linear_model import ElasticNet

In [29]:
enet_model = ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42)

# Entrenar el modelo para cada variable objetivo
r2_scores = {}  # Un diccionario para almacenar los resultados R^2

for variable_objetivo in y.columns:
    y_actual = y[variable_objetivo]  # Selecciona la variable objetivo actual
    enet_model.fit(X, y_actual)  # Entrena el modelo para la variable objetivo actual
    y_pred = enet_model.predict(X)  # Realiza predicciones
    
    # Calcula el coeficiente de determinación (R^2) para la variable objetivo actual
    r2 = r2_score(y_actual, y_pred)
    r2_scores[variable_objetivo] = r2

    # Imprime el resultado R^2 para la variable objetivo actual
    print(f'R^2 para {variable_objetivo}: {r2}')

R^2 para BEN: 0.20562401134708963
R^2 para CH4: 0.1634881278720841
R^2 para CO: 0.11820146981801116
R^2 para EBE: 0.009885208570561632
R^2 para NMHC: 0.012692283421895989
R^2 para NO: 0.30280050157855964
R^2 para NO2: 0.39690668718793387
R^2 para NOX: 0.36572687923658553
R^2 para O3: 0.6480957709025448
R^2 para PM10: 0.16412100512205607
R^2 para PM25: 0.1602900163699934
R^2 para SO2: 0.14807696049729524
R^2 para TCH: 0.15391076856689412
R^2 para TOL: 0.17491207403512743


In [31]:
from sklearn.neural_network import MLPRegressor

In [33]:
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)

# Entrenar el modelo para cada variable objetivo
r2_scores = {}  # Un diccionario para almacenar los resultados R^2

for variable_objetivo in y.columns:
    y_actual = y[variable_objetivo]  # Selecciona la variable objetivo actual
    mlp_model.fit(X, y_actual)  # Entrena el modelo para la variable objetivo actual
    y_pred = mlp_model.predict(X)  # Realiza predicciones
    
    # Calcula el coeficiente de determinación (R^2) para la variable objetivo actual
    r2 = r2_score(y_actual, y_pred)
    r2_scores[variable_objetivo] = r2

    # Imprime el resultado R^2 para la variable objetivo actual
    print(f'R^2 para {variable_objetivo}: {r2}')

R^2 para BEN: 0.0137709974351381
R^2 para CH4: -0.08414105082582601
R^2 para CO: -3.830349859631978
R^2 para EBE: -0.5289086188595384
R^2 para NMHC: -77.51800095576554
R^2 para NO: 0.5405587181196572
R^2 para NO2: 0.6001480669659219
R^2 para NOX: 0.5862087908603079
R^2 para O3: 0.7585339738701322
R^2 para PM10: 0.4556694862825693
R^2 para PM25: 0.34504097912799625
R^2 para SO2: 0.36235330527093823
R^2 para TCH: 0.112100306272792
R^2 para TOL: 0.12952492072529986


In [18]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

In [19]:
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# Entrenar el modelo para cada variable objetivo
r2_scores = {}  # Un diccionario para almacenar los resultados R^2

for variable_objetivo in y.columns:
    y_actual = y[variable_objetivo]  # Selecciona la variable objetivo actual
    gb_model.fit(X, y_actual)  # Entrena el modelo para la variable objetivo actual
    y_pred = gb_model.predict(X)  # Realiza predicciones
    
    # Calcula el coeficiente de determinación (R^2) para la variable objetivo actual
    r2 = r2_score(y_actual, y_pred)
    r2_scores[variable_objetivo] = r2

    # Imprime el resultado R^2 para la variable objetivo actual
    print(f'R^2 para {variable_objetivo}: {r2}')

R^2 para BEN: 0.7963613422615344
R^2 para CH4: 0.6767192627870107
R^2 para CO: 0.6119757006696591
R^2 para EBE: 0.6375490877108249
R^2 para NMHC: 0.56655621080933
R^2 para NO: 0.7493710502846108
R^2 para NO2: 0.7298829149296053
R^2 para NOX: 0.7537239315735569
R^2 para O3: 0.8356339878018723
R^2 para PM10: 0.7286200105585453
R^2 para PM25: 0.7225877922585737
R^2 para SO2: 0.5838856871341757
R^2 para TCH: 0.6683476780920425
R^2 para TOL: 0.6584535340207708


In [20]:
from sklearn.ensemble import BaggingRegressor

In [21]:
bagging_model = BaggingRegressor(base_estimator=None, n_estimators=100, random_state=42)

# Entrenar el modelo para cada variable objetivo
r2_scores = {}  # Un diccionario para almacenar los resultados R^2

for variable_objetivo in y.columns:
    y_actual = y[variable_objetivo]  # Selecciona la variable objetivo actual
    bagging_model.fit(X, y_actual)  # Entrena el modelo para la variable objetivo actual
    y_pred = bagging_model.predict(X)  # Realiza predicciones
    
    # Calcula el coeficiente de determinación (R^2) para la variable objetivo actual
    r2 = r2_score(y_actual, y_pred)
    r2_scores[variable_objetivo] = r2

    # Imprime el resultado R^2 para la variable objetivo actual
    print(f'R^2 para {variable_objetivo}: {r2}')

R^2 para BEN: 0.9680130538614573
R^2 para CH4: 0.9801937937721128
R^2 para CO: 0.936992257163168
R^2 para EBE: 0.9261042636906808
R^2 para NMHC: 0.9703623457673375
R^2 para NO: 0.9595322594936018
R^2 para NO2: 0.9636273815551285
R^2 para NOX: 0.9639967319962874
R^2 para O3: 0.9812297539582301
R^2 para PM10: 0.9676738587649485
R^2 para PM25: 0.9711468207265171
R^2 para SO2: 0.9383778311893712
R^2 para TCH: 0.9795690320398264
R^2 para TOL: 0.9539131792285681


In [23]:
from sklearn.ensemble import AdaBoostRegressor

In [25]:
adaboost_regressor = AdaBoostRegressor(n_estimators=100, random_state=42)

# Entrenar el modelo para cada variable objetivo
r2_scores = {}  # Un diccionario para almacenar los resultados R^2

for variable_objetivo in y.columns:
    y_actual = y[variable_objetivo]  # Selecciona la variable objetivo actual
    adaboost_regressor.fit(X, y_actual)  # Entrena el modelo para la variable objetivo actual
    y_pred = adaboost_regressor.predict(X)  # Realiza predicciones
    
    # Calcula el coeficiente de determinación (R^2) para la variable objetivo actual
    r2 = r2_score(y_actual, y_pred)
    r2_scores[variable_objetivo] = r2

    # Imprime el resultado R^2 para la variable objetivo actual
    print(f'R^2 para {variable_objetivo}: {r2}')

R^2 para BEN: -0.10053095882589691
R^2 para CH4: 0.39639417694562273
R^2 para CO: 0.06803158647547025
R^2 para EBE: 0.06505756999456547
R^2 para NMHC: -0.39440982740872643
R^2 para NO: -0.043157398762597055
R^2 para NO2: 0.38985004685609637
R^2 para NOX: 0.10753230537015668
R^2 para O3: 0.6987619203241725
R^2 para PM10: 0.03341218930332013
R^2 para PM25: -0.42231081341517873
R^2 para SO2: 0.2602654728343524
R^2 para TCH: 0.3957387129815888
R^2 para TOL: -4.591608739659476


In [26]:
from sklearn.linear_model import Ridge

In [27]:
modelo_ridge = Ridge(alpha=1.0, solver='auto')

# Entrenar el modelo para cada variable objetivo
r2_scores = {}  # Un diccionario para almacenar los resultados R^2

for variable_objetivo in y.columns:
    y_actual = y[variable_objetivo]  # Selecciona la variable objetivo actual
    modelo_ridge.fit(X, y_actual)  # Entrena el modelo para la variable objetivo actual
    y_pred = modelo_ridge.predict(X)  # Realiza predicciones
    
    # Calcula el coeficiente de determinación (R^2) para la variable objetivo actual
    r2 = r2_score(y_actual, y_pred)
    r2_scores[variable_objetivo] = r2

    # Imprime el resultado R^2 para la variable objetivo actual
    print(f'R^2 para {variable_objetivo}: {r2}')

R^2 para BEN: 0.47494440073820454
R^2 para CH4: 0.19236341371805876
R^2 para CO: 0.3513004971828235
R^2 para EBE: 0.05695377355327913
R^2 para NMHC: 0.06742061324574822
R^2 para NO: 0.3403702801790409
R^2 para NO2: 0.4776186793807735
R^2 para NOX: 0.4175059619002125
R^2 para O3: 0.6862901419309391
R^2 para PM10: 0.18251240482859543
R^2 para PM25: 0.22134774137569668
R^2 para SO2: 0.152699332255119
R^2 para TCH: 0.18059067645256677
R^2 para TOL: 0.29297177047609413


In [28]:
from sklearn.linear_model import Lasso

In [29]:
modelo_lasso = Lasso(alpha=1.0)

# Entrenar el modelo para cada variable objetivo
r2_scores = {}  # Un diccionario para almacenar los resultados R^2

for variable_objetivo in y.columns:
    y_actual = y[variable_objetivo]  # Selecciona la variable objetivo actual
    modelo_lasso.fit(X, y_actual)  # Entrena el modelo para la variable objetivo actual
    y_pred = modelo_lasso.predict(X)  # Realiza predicciones
    
    # Calcula el coeficiente de determinación (R^2) para la variable objetivo actual
    r2 = r2_score(y_actual, y_pred)
    r2_scores[variable_objetivo] = r2

    # Imprime el resultado R^2 para la variable objetivo actual
    print(f'R^2 para {variable_objetivo}: {r2}')

R^2 para BEN: 0.1636963751769701
R^2 para CH4: 0.09977888116625366
R^2 para CO: 0.10803316887223069
R^2 para EBE: 0.003980144319569168
R^2 para NMHC: 0.0
R^2 para NO: 0.32868557195966663
R^2 para NO2: 0.46453558442925225
R^2 para NOX: 0.41541187353437015
R^2 para O3: 0.6785630227687642
R^2 para PM10: 0.1598033288862265
R^2 para PM25: 0.13781238068497803
R^2 para SO2: 0.14328480353626238
R^2 para TCH: 0.0991648409991247
R^2 para TOL: 0.15995399151273282


## BUSQUEDA DE METAPARAMETROS

In [28]:
from sklearn.model_selection import GridSearchCV

In [45]:
param_grid = {
    'base_estimator': [None],
    'n_estimators': [50, 100, 150],
    'max_depth': [5,10,15]
    'random_state': [42]
}

In [46]:
# Búsqueda de meta-parámetros:
grid_search = GridSearchCV(estimator=bagging_model, param_grid=param_grid, cv=5)

In [47]:
# Ajustar el modelo a los datos
grid_search.fit(X,y)

GridSearchCV(cv=5,
             estimator=BaggingRegressor(n_estimators=100, random_state=42),
             param_grid={'base_estimator': [None],
                         'n_estimators': [50, 100, 150], 'random_state': [42]})

In [48]:
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

Mejores hiperparámetros encontrados:
{'base_estimator': None, 'n_estimators': 150, 'random_state': 42}


In [49]:
best_model = grid_search.best_estimator_
best_model.fit(X, y)

BaggingRegressor(n_estimators=150, random_state=42)

In [50]:
y_pred = best_model.predict(X)

In [51]:
r2 = r2_score(y, y_pred)
print(f"MSE en el conjunto de entrenamiento: {r2}")

MSE en el conjunto de entrenamiento: 0.9603507188269377
